In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Add
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.layers import Conv1D, Activation, Multiply

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score


In [2]:
df_raw = pd.read_csv("Datasetforimplementation.csv")

In [3]:
df_raw.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

In [4]:
Encoder = "Label Encoder"
# Encoder = "One-hot Encoder"
# OverSamplingTecnique = ""
# OverSamplingTecnique = "SMOTE"
# OverSamplingTecnique = "SMOTE-Tomek"
OverSamplingTecnique = "SMOTE-Enn"
filter_size=5
number_of_filter=128
flatten_layer_exist=True
Model_Name="SE Block"
# Model_Name="Basic Channel Attention"

In [5]:
if Encoder == "Label Encoder":
  print("Applying Label Encoder")
  df_final = df_raw.copy()
  le = LabelEncoder()

  text_data_features = ['Geography', 'Gender']

  print('Label Encoder Transformation')
  for i in text_data_features :
      df_final[i] = le.fit_transform(df_final[i])
      print(i,' : ',df_final[i].unique(),' = ',le.inverse_transform(df_final[i].unique()))



  X = df_final.drop(['Exited'], axis=1).copy()
  Y = df_final['Exited'].copy().astype(int)

Applying Label Encoder
Label Encoder Transformation
Geography  :  [0 2 1]  =  ['France' 'Spain' 'Germany']
Gender  :  [0 1]  =  ['Female' 'Male']


In [6]:
if Encoder == "One-hot Encoder":
  print("Applying One-hot Encoder")

  # One-hot encode categorical columns
  categorical_columns = ['Geography', 'Gender']

  encoder = OneHotEncoder()
  encoded_features = encoder.fit_transform(df_raw[categorical_columns]).toarray()

  # Combine one-hot encoded features with numerical features
  numerical_features = df_raw.drop(categorical_columns + ['Exited'], axis=1)
  X = np.hstack((encoded_features, numerical_features))

  # Manually encode 'Churn' column
  # df_raw['Churn'] = df_raw['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)
  # Extract the target variable Y
  Y = df_raw['Exited'].values


  # Ensure all data is in float format
  X = X.astype(float)
  Y = Y.astype(int)

In [7]:
scaler = StandardScaler()
X_resampled_scaled = scaler.fit_transform(X)
X=X_resampled_scaled
Y=Y

In [8]:
# if OverSamplingTecnique == "SMOTE":
#   print("Applying SMOTE")
#   smote = SMOTE()

#   X_resampled, y_resampled = smote.fit_resample(X, Y)
#   scaler = StandardScaler()
#   X_resampled_scaled = scaler.fit_transform(X_resampled)
#   X=X_resampled_scaled
#   Y=y_resampled

In [9]:
# if OverSamplingTecnique == "SMOTE-Tomek":
#   print("Applying SMOTE-Tomek")

#   smote_tomek = SMOTETomek()
#   X_resampled, y_resampled = smote_tomek.fit_resample(X, Y)
#   scaler = StandardScaler()
#   X_resampled_scaled = scaler.fit_transform(X_resampled)
#   X=X_resampled_scaled
#   Y=y_resampled

In [10]:
if OverSamplingTecnique == "SMOTE-Enn":
  print("Applying SMOTE-Enn")

  smote_enn = SMOTEENN()
  X_resampled, y_resampled = smote_enn.fit_resample(X, Y)
  scaler = StandardScaler()
  X_resampled_scaled = scaler.fit_transform(X_resampled)
  X=X_resampled_scaled
  Y=y_resampled

Applying SMOTE-Enn


In [11]:
import torch
import torch.nn as nn

class ChannelAttention(nn.Module):
    def __init__(self, in_channels, reduction=8):
        super(ChannelAttention, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(in_channels, in_channels // reduction),
            nn.ReLU(),
            nn.Linear(in_channels // reduction, in_channels),
            nn.Sigmoid()
        )

    def forward(self, x):
        if x.dim() == 4:  # 4D input (batch, channels, height, width) -> CNN
            batch_size, channels, height, width = x.shape
            avg_pool = torch.mean(x, dim=[2, 3])  # Global Average Pooling
        elif x.dim() == 3:  # 3D input (batch, channels, length) -> 1D CNN
            batch_size, channels, length = x.shape
            avg_pool = torch.mean(x, dim=2)  # Global Average Pooling along length
        else:
            raise ValueError(f"Expected input to have 3 or 4 dimensions, but got {x.shape}")

        attn_weights = self.fc(avg_pool)
        attn_weights = attn_weights.unsqueeze(-1)  # Expand for broadcasting

        return x * attn_weights  # Apply attention weights


# Define Spatial Attention Layer
class SpatialAttention(nn.Module):
    def __init__(self):
        super(SpatialAttention, self).__init__()
        self.conv = nn.Conv1d(2, 1, kernel_size=3, padding=1, bias=False)

    def forward(self, x):
        max_pool = torch.max(x, dim=1, keepdim=True)[0]  # Max pooling along channels
        avg_pool = torch.mean(x, dim=1, keepdim=True)  # Average pooling along channels
        attn_weights = torch.cat([max_pool, avg_pool], dim=1)
        attn_weights = self.conv(attn_weights)
        attn_weights = torch.sigmoid(attn_weights)
        return x * attn_weights


# Define Residual Block
class ResidualBlock(nn.Module):
    def __init__(self, channels, kernel_size):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv1d(channels, channels, kernel_size, padding=kernel_size//2)
        self.conv2 = nn.Conv1d(channels, channels, kernel_size, padding=kernel_size//2)
        self.relu = nn.ReLU()

    def forward(self, x):
        shortcut = x
        x = self.relu(self.conv1(x))
        x = self.conv2(x)
        return self.relu(x + shortcut)

In [12]:
# Define Basic Channel Attention
class BasicChannelAttention(nn.Module):
    def __init__(self, channels, reduction_ratio=8):
        super(BasicChannelAttention, self).__init__()
        self.shared_conv1 = nn.Conv1d(1, channels // reduction_ratio, kernel_size=1, padding=0)
        self.shared_conv2 = nn.Conv1d(channels // reduction_ratio, 1, kernel_size=1, padding=0)

    def forward(self, x):
        avg_pool = torch.mean(x, dim=1, keepdim=True)
        max_pool = torch.max(x, dim=1, keepdim=True)[0]

        attn_avg = self.shared_conv2(F.relu(self.shared_conv1(avg_pool)))
        attn_max = self.shared_conv2(F.relu(self.shared_conv1(max_pool)))

        attention = torch.sigmoid(attn_avg + attn_max)
        return x * attention

In [13]:
# Define Model
class AttentionModel(nn.Module):
    def __init__(self, input_size, num_filters, kernel_size, model_name, use_flatten):
        super(AttentionModel, self).__init__()
        self.conv1 = nn.Conv1d(1, num_filters, kernel_size, padding=kernel_size//2)
        self.res_block1 = ResidualBlock(num_filters, kernel_size)
        self.res_block2 = ResidualBlock(num_filters, kernel_size)

        if model_name == "SE Block":
            self.channel_attn1 = ChannelAttention(num_filters)
            self.channel_attn2 = ChannelAttention(num_filters)
        else:
            self.channel_attn1 = BasicChannelAttention(num_filters)
            self.channel_attn2 = BasicChannelAttention(num_filters)

        self.spatial_attn1 = SpatialAttention()
        self.spatial_attn2 = SpatialAttention()

        self.use_flatten = use_flatten

        if use_flatten:
            self.fc = nn.Sequential(
                nn.Flatten(),
                nn.Linear(num_filters * input_size, num_filters),
                nn.ReLU(),
                nn.Dropout(0.5),
                nn.Linear(num_filters, 1),
                nn.Sigmoid()
            )
        else:
            self.global_pool = nn.AdaptiveMaxPool1d(1)
            self.fc = nn.Sequential(
                nn.Linear(num_filters, num_filters),
                nn.ReLU(),
                nn.Dropout(0.5),
                nn.Linear(num_filters, 1),
                nn.Sigmoid()
            )

    def forward(self, x):
        x = self.conv1(x)
        x = self.res_block1(x)
        x = self.channel_attn1(x)
        x = self.spatial_attn1(x)
        x = self.res_block2(x)
        x = self.channel_attn2(x)
        x = self.spatial_attn2(x)

        if self.use_flatten:
            x = self.fc(x)
        else:
            x = self.global_pool(x).squeeze(-1)
            x = self.fc(x)

        return x

In [14]:
def kfold(X, Y, filter_size, num_filters, use_flatten, model_name, epochs=30, batch_size=32, num_folds=10):
    print("Applying K-fold")
    skf = StratifiedKFold(n_splits=num_folds, shuffle=True)

    metrics = {
        "accuracy": [], "precision": [], "recall": [], "f1": [], "mcc": [], "auc_roc": []
    }

    # Ensure X and Y are NumPy arrays
    X = np.array(X)
    Y = np.array(Y)

    for fold, (train_idx, test_idx) in enumerate(skf.split(X, Y), 1):
        print(f"Fold {fold}/{num_folds}:")

        # Convert to PyTorch tensors
        X_train = torch.tensor(X[train_idx], dtype=torch.float32).unsqueeze(1)
        Y_train = torch.tensor(Y[train_idx], dtype=torch.float32).unsqueeze(1)
        X_test = torch.tensor(X[test_idx], dtype=torch.float32).unsqueeze(1)
        Y_test = torch.tensor(Y[test_idx], dtype=torch.float32).unsqueeze(1)

        train_dataset = TensorDataset(X_train, Y_train)
        test_dataset = TensorDataset(X_test, Y_test)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        # Initialize model
        model = AttentionModel(input_size=X_train.shape[2], num_filters=num_filters, kernel_size=filter_size, model_name=model_name, use_flatten=use_flatten)
        optimizer = optim.Adam(model.parameters(), lr=1e-5)
        loss_fn = nn.BCELoss()

        # Train the model
        model.train()
        for epoch in range(epochs):
            for inputs, labels in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs).squeeze()
                loss = loss_fn(outputs, labels.squeeze())
                loss.backward()
                optimizer.step()

        # Evaluate the model
        model.eval()
        y_true, y_pred = [], []
        with torch.no_grad():
            for inputs, labels in test_loader:
                outputs = model(inputs).squeeze()
                preds = (outputs >= 0.5).float()
                y_true.extend(labels.squeeze().tolist())
                y_pred.extend(preds.tolist())

        # Compute metrics
        metrics["accuracy"].append(accuracy_score(y_true, y_pred))
        metrics["precision"].append(precision_score(y_true, y_pred))
        metrics["recall"].append(recall_score(y_true, y_pred))
        metrics["f1"].append(f1_score(y_true, y_pred))
        metrics["mcc"].append(matthews_corrcoef(y_true, y_pred))
        metrics["auc_roc"].append(roc_auc_score(y_true, y_pred))

    # Print average scores
    for key, values in metrics.items():
        print(f"Average {key.capitalize()}: {np.mean(values):.4f}")


In [15]:
kfold(X, Y, filter_size=3, num_filters=64, use_flatten=True, model_name="SE Block")

Applying K-fold
Fold 1/10:
Fold 2/10:
Fold 3/10:
Fold 4/10:
Fold 5/10:
Fold 6/10:
Fold 7/10:
Fold 8/10:
Fold 9/10:
Fold 10/10:
Average Accuracy: 0.8711
Average Precision: 0.8915
Average Recall: 0.8791
Average F1: 0.8852
Average Mcc: 0.7385
Average Auc_roc: 0.8699
